In [1]:
import os
import sys
from anthropic import Anthropic, AnthropicError

notebook_path = os.getcwd()
project_path = os.path.abspath(os.path.join(notebook_path, '..','..'))
print(f"Project path: {project_path}")
# Add the project path to sys.path
sys.path.append(project_path)

Project path: c:\Users\dalej\Documents\_Coding\DragonRegen


In [2]:
from src.AIGuardian.ActionsFrame.DataColumnRefiner import DataColumnRefiner

# Inputs
action_parameters = {
                "table_name": 'members',
                "purpose": "To track all the information about members of the gym.",
                "column_name": 'membership',
                "description": 'The kind of membership the member has.',
            }

test_action = DataColumnRefiner(input_params=action_parameters)
print(test_action)

In [3]:
test_action.run(user_prompt="")

create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <column_information> to fill in JSON values and add additional metadata for the column.\n        <column_information>\n        Table Purpose: To track all the information about members of the gym.\n        Column Info: "membership": "The kind of membership the member has."\n        </column_information>\n        Please provide the response in a JSON format like the <JSON_Template> below. There are examples below at <Examples>.\n        <JSON_Template>\n        {"name": "example_column", "type": "Integer", "nullable": true, "metadata": {"description": "Place the description of the column here.", "unique_fl": true, "default_value": null}}\n        </JSON_Template>\n\n        <Examples>\n        Example 1:\n        Purpose: "This table is used to store user information."\n        Column I

In [5]:
print(test_action.response)

Message(id='msg_01YDq8i8diSFfpcgQP36n2Ut', content=[TextBlock(citations=None, text='\n  "name": "membership",\n  "type": "String",\n  "nullable": false,\n  "metadata": {\n    "description": "The kind of membership the member has.",\n    "unique_fl": false,\n    "default_value": null,\n    "possible_values": ["Basic", "Premium", "Gold", "Platinum", "Family", "Student", "Senior"],\n    "category": "Membership Information",\n    "validation_rules": "Must be one of the predefined membership types"\n  }\n}\n', type='text')], model='claude-3-7-sonnet-20250219', role='assistant', stop_reason='stop_sequence', stop_sequence='</JSON_Template>', type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=295, output_tokens=129))


## Test Data Struct

In [ ]:
# from src.AIGuardian.ActionsFrame.DataStructCreate import DataStructCreate

# datastruct_action = DataStructCreate()
# prompt = """Create a data structure for a gym membership system."""
# datastruct_action.run(prompt)
# print(datastruct_action.response)

SQL: INSERT INTO ai_actions_log (action_name, action_version, parent_action_id, group_action_id, description, sequence_number, created_dt, updated_dt, status, metadata) 
        OUTPUT INSERTED.action_id
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        
create_params: {'max_tokens': 10000, 'temperature': 0.1, 'stop_sequences': ['</JSON_Template>'], 'model': 'claude-3-7-sonnet-20250219', 'messages': [{'role': 'user', 'content': 'Please take the following <business_requirements> and create the table list and schema for the the request.\n        <business_requirements>\n        Create a data structure for a gym membership system.\n        </business_requirements>\n        Please provide the response in a JSON format like the <JSON_Template> below:\n        <JSON_Template>\n        {"table_name_1": {\n            "purpose": "short description of tables usage in the architecture",\n            "fields":{"column_name_1": "unique ID, short description of column",\n            , "column_n